# Data Preparation
In the first step we load the data set; filter it based on the occupational code and clean it.

In [2]:
#importing the necessary libraries

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [5]:
#loading the dataset
data = pd.read_csv('morg-2014-emp.csv', low_memory= False)
data.head()

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
0,3,2600310997690,January,AL,3151.6801,1692.00,40,43,1,NaN,...,0,0,"Native, Born In US",63,Employment services (5613),630,"Private, For Profit",No,No,Employed-At Work
1,5,75680310997590,January,AL,3457.1138,450.00,40,41,2,NaN,...,2,6,"Native, Born In US",63,Outpatient care centers (6214),5400,"Private, For Profit",No,No,Employed-Absent
2,6,75680310997590,January,AL,3936.9110,1090.00,60,41,2,NaN,...,2,6,"Native, Born In US",63,Motor vehicles and motor vehicle equipment man...,8140,"Private, For Profit",No,No,Employed-At Work
3,10,179140131100930,January,AL,3288.3640,769.23,40,40,1,NaN,...,2,4,"Native, Born In US",63,"**Publishing, except newspapers and software (...",8255,"Private, For Profit",Yes,NaN,Employed-At Work
4,11,179140131100930,January,AL,3422.8500,826.92,40,43,1,NaN,...,2,4,"Native, Born In US",63,"Banking and related activities (521, 52211,52219)",5940,"Private, For Profit",No,No,Employed-At Work


## Filtering the data based on the occupational codes
We are going to continue with **__Arts, Design, Entertainment, Sports, and Media Occupations__** in this assignment.

In [7]:
#filtering data based on the Occupational code
filtered_data = data[(data['occ2012'] >= 2600) & (data['occ2012'] <= 2920)]

* Checking the data types to make the necessary adjustments

In [8]:
filtered_data.dtypes

Unnamed: 0      int64
hhid            int64
intmonth       object
stfips         object
weight        float64
earnwke       float64
uhours          int64
grade92         int64
race            int64
ethnic        float64
age             int64
sex             int64
marital         int64
ownchild        int64
chldpres        int64
prcitshp       object
state          object
ind02          object
occ2012         int64
class          object
unionmme       object
unioncov       object
lfsr94         object
dtype: object

In [11]:
# converting the categorical columns from object to categorical type

cat_cols = [
    'intmonth','stfips','prcitshp', 'ind02','class', 'unionmme','unioncov',
    'lfsr94', 'race', 'ethnic', 'sex', 'marital','occ2012' 
]
for col in cat_cols:
    filtered_data[col] = pd.Categorical(filtered_data[col])

C:\Users\PA\AppData\Local\Temp\ipykernel_15412\2183723912.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[col] = pd.Categorical(filtered_data[col])


# Missing values, checking and handling
First, let's see which variables include missing values and how can we get rid of them.

In [12]:
missing_values = filtered_data.isnull().sum()
missing_values

Unnamed: 0       0
hhid             0
intmonth         0
stfips           0
weight           0
earnwke          0
uhours           0
grade92          0
race             0
ethnic        2107
age              0
sex              0
marital          0
ownchild         0
chldpres         0
prcitshp         0
state            0
ind02            0
occ2012          0
class            0
unionmme         0
unioncov       134
lfsr94           0
dtype: int64

* Only 2 variables have misisng values and for these 2, it is plausible to replace the missing values with the modes.

In [16]:
import warnings

# filter out FutureWarning (with the help of github copilot, to avoid the warning)
warnings.filterwarnings('ignore', category=FutureWarning)

# fill missing values with mode
filtered_data['ethnic'].fillna(filtered_data['ethnic'].mode()[0], inplace=True)
filtered_data['unioncov'].fillna(filtered_data['unioncov'].mode()[0], inplace=True)


C:\Users\PA\AppData\Local\Temp\ipykernel_15412\3100252707.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['ethnic'].fillna(filtered_data['ethnic'].mode()[0], inplace=True)
C:\Users\PA\AppData\Local\Temp\ipykernel_15412\3100252707.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['unioncov'].fillna(filtered_data['unioncov'].mode()[0], inplace=True)


In [17]:
# Creating a new variable for hourly wage (with the help of github copilot, to avoid the warning)
filtered_data['hourly_wage'] = filtered_data['earnwke'].div(filtered_data['uhours'])


C:\Users\PA\AppData\Local\Temp\ipykernel_15412\2561687977.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['hourly_wage'] = filtered_data['earnwke'].div(filtered_data['uhours'])


### Converting the categorical variables to dummies
* When we convert a categorical variable to a set of dummies, the sufficient number of dummies is the number of categories minus 1. This is the reason that we drop the first category.
* Also, after creating the set of dummies we exclude the original variable.

In [18]:
# Converting the categorical variables to dummies
filtered_data = pd.get_dummies(filtered_data, columns = cat_cols, drop_first= True )

In [31]:
drop_cols = cat_cols + ['Unnamed: 0','hhid', 'earnwke', 'uhours','state']

In [32]:
keep_cols= [i for i in filtered_data.columns if i not in drop_cols]
df = filtered_data[keep_cols].copy()
df = df.reset_index(drop=True)

* Storing the cleaned data in a new csv file

In [35]:
df.to_csv('cleaned_filtered_numeric_data.csv')

In [34]:
df

,weight,grade92,age,ownchild,chldpres,hourly_wage,intmonth_August,intmonth_December,intmonth_February,intmonth_January,...,occ2012_2800,occ2012_2810,occ2012_2825,occ2012_2830,occ2012_2840,occ2012_2850,occ2012_2860,occ2012_2900,occ2012_2910,occ2012_2920
0,3202.3739,40,23,0,0,8.350000,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,9206.4248,40,23,0,0,7.750000,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1533.1598,43,35,2,8,22.200000,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1616.3952,40,27,1,1,17.500000,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1833.6081,37,17,0,0,13.500000,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2310,3374.2812,43,32,1,1,19.230750,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2311,3677.2160,39,50,1,4,10.357143,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2312,3814.6828,43,57,0,0,23.397222,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2313,325.7699,41,36,2,8,38.450000,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
